In [ ]:
import tensorflow as tf
import numpy as np

from PIL import Image
import os
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
"""
    Original GAN paper -- 'https://arxiv.org/abs/1406.2661'
    But these GAN are hard to train.

    So some tried and tested rule of thumbs shuld be followed to get results.
    This Notebook follow these hacks -- 'https://github.com/soumith/ganhacks'

    This Notebook is based on DCGANs paper -- 'https://arxiv.org/abs/1511.06434'

"""

In [ ]:
"""
  FOR MNIST DATASET - DISCRIMINATOR MODEL
"""

inputs = tf.keras.layers.Input((28,28,1))

out = tf.keras.layers.Conv2D( 64,(5,5),strides=(2,2), padding='same' )(inputs)
out = tf.keras.layers.LeakyReLU()(out)
out = tf.keras.layers.Dropout(0.3)(out)

out = tf.keras.layers.Conv2D( 128,(5,5),strides=(2,2) , padding='same' )(out)
out = tf.keras.layers.LeakyReLU()(out)
out = tf.keras.layers.Dropout(0.3)(out)

out = tf.keras.layers.Flatten()(out)
outputs = tf.keras.layers.Dense( 1 )(out)

D_model=0
D_model = tf.keras.Model(inputs=inputs,outputs=outputs)
D_model.summary()

In [ ]:
"""
  FOR MNIST DATASET - GENERATOR MODEL
"""


inputs = tf.keras.layers.Input((100,))

out = tf.keras.layers.Dense(7*7*256)(inputs)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.LeakyReLU()(out)

out = tf.keras.layers.Reshape(target_shape=(7,7,256))(out)

out = tf.keras.layers.Conv2DTranspose(128,(5,5),strides=(1,1),padding="same",use_bias=False)(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.LeakyReLU()(out)

out = tf.keras.layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding="same",use_bias=False)(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.LeakyReLU()(out)

outputs = tf.keras.layers.Conv2DTranspose(1,(5,5),strides=(2,2),use_bias=False,activation='tanh',padding="same")(out)

G_model = 0
G_model=tf.keras.Model(inputs=inputs,outputs=outputs)
G_model.summary()

In [ ]:
try_gen = G_model( tf.random.normal([1,100]) , training=False )
print(try_gen.shape)
show = plt.imshow( try_gen[0,:,:,0]*127.5+127.5 ,cmap='gray' )
plt.show()

In [ ]:
import tensorflow_datasets as tfds
raw_data,info = tfds.load('mnist',split=['train','test'],with_info=True)

print(info)

In [ ]:
train_data,test_data = raw_data[0],raw_data[1]

def map_func(data):

  img = ( tf.cast( data['image'] , tf.float32 )-127.5 )/127.5
  return img

train_data = train_data.map( map_func )

train_data = train_data.shuffle(10000)
train_data = train_data.batch(256)

for one_b in train_data.take(1):
  show = plt.imshow(one_b[0,:,:,0],cmap='gray')
  plt.show()
  print( one_b.shape )
  print( one_b[0,:,:,0])

In [ ]:
seed = tf.random.normal([ 16 , 100 ])

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
#********************
batch_size = 256
l_rate = 0.001
ctr=0
epochs=20
# k -factor
k=1
#********************

d_optimizer = tf.keras.optimizers.Adam( l_rate )
g_optimizer = tf.keras.optimizers.Adam( l_rate )

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# tf abracted implementation. ( Its a bit Smooth !)
# def discriminator_loss(real_output, fake_output):
#     real_loss = cross_entropy(tf.ones_like(real_output), real_output)
#     fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
#     total_loss = real_loss + fake_loss
#     return total_loss

# def generator_loss(fake_output):
#     return cross_entropy(tf.ones_like(fake_output), fake_output)

# My implementation
def cost_gen(pred_prob):

    delta = 1e-8
    loss = (-1.0/pred_prob.shape[0])*tf.reduce_sum( tf.math.log(  1-tf.nn.sigmoid(pred_prob) ) + delta )
    return loss+delta

def cost_data(pred_prob):

    delta = 1e-8
    loss = (-1.0/pred_prob.shape[0])*tf.reduce_sum( tf.math.log( tf.nn.sigmoid(pred_prob) ) +delta )
    return loss+delta

def cost_fn_g(pred_prob):
    delta = 1e-8
    loss = (-1.0/pred_prob.shape[0])*tf.reduce_sum( tf.math.log( tf.nn.sigmoid(pred_prob) ) + delta )
    return loss+delta

with tf.device('/device:GPU:0'):

  for e in range(epochs):
    
    print("********* EPOCH :: {}".format(e))

    # display.clear_output(wait=True)
    generate_and_save_images(G_model,
                             e + 1,
                             seed)

    g_cost = 0

    for batch_true in train_data:
      
      batch_noise = tf.random.normal( (batch_size,100 ) )
      
      with tf.GradientTape(persistent=True) as tape:

        pred_true = tf.squeeze( D_model( batch_true , training=True ) )
        batch_false = G_model(batch_noise, training=True)
        pred_false = tf.squeeze( D_model(batch_false, training=True) )

        # For tf abstracted implementation
        # d_cost = discriminator_loss(pred_true,pred_false)
        # g_cost = generator_loss(pred_false)

        cost_true = cost_data(pred_true)
        cost_false = cost_gen(pred_false)

        d_cost = (cost_true + cost_false)
        g_cost = cost_fn_g(pred_false)


      if ctr%100==0:
        print("D_Loss is === >  {}".format(d_cost))
        print("G_Loss is === >  {}".format(g_cost))

      ctr = ctr+1
      
      if ctr%k==0:
        d_grads = tape.gradient( d_cost,D_model.trainable_variables )
        d_optimizer.apply_gradients(zip(d_grads,D_model.trainable_variables))

      g_grads = tape.gradient( g_cost,G_model.trainable_variables )
      g_optimizer.apply_gradients(zip(g_grads,G_model.trainable_variables))
    
